## Code source du modèle

### Avertissement

⚠️ **Veuillez effectuer l'étape de configuration avec soin**

Il est essentiel de compléter l'étape de configuration correctement pour éviter toute erreur. Veuillez vous assurer de respecter les points suivants :

- **Avant tout push vers GitHub,** assurez-vous de ne pas laisser d'informations sensibles ou compromettantes dans votre code.
- **Le dépôt doit être cloné dans le bon répertoire sur Google Drive** pour garantir que les fichiers sont correctement synchronisés.
- **Git doit être correctement configuré** avant d'effectuer toute action sur le dépôt, telles que `git add`, `git commit`, etc.

---

⚠️ **Ne jamais exposer votre `GITHUB_TOKEN`**.

Il est impératif que votre `GITHUB_TOKEN` soit gardé secret. Ne le partagez **jamais** dans des dépôts publics, des fichiers non sécurisés ou des endroits où des tiers pourraient y accéder. Toute fuite de votre token pourrait entraîner des accès non autorisés à vos projets et à vos données privées.

---

### Conseil : Utilisation de `!bash -c` pour les commandes Linux dans Google Colab

Lorsque vous exécutez des commandes Linux dans Google Colab, il est recommandé d'utiliser `!bash -c` pour enchaîner plusieurs commandes dans un même environnement. Cela vous permet de garder toutes les commandes dans un même contexte de shell. Par exemple, si vous voulez changer de répertoire et exécuter plusieurs commandes, vous pouvez utiliser la commande suivante :

```bash
!bash -c "cd /content/drive/MyDrive/SAE_S5 && git status"
```

### Explication du bloc "Conseil" :

- **Utilisation de `!bash -c`** : Quand tu souhaites enchaîner plusieurs commandes dans Google Colab dans un même shell (par exemple, changer de répertoire avec `cd` puis exécuter d'autres commandes comme `git status`), tu peux utiliser `!bash -c`. Cela te permet d'éviter de devoir ouvrir plusieurs cellules ou d'essayer de combiner les commandes de manière incorrecte.
  
- **Syntaxe** : La commande `!bash -c "commandes"` permet d'exécuter plusieurs commandes Linux dans un même contexte, ce qui garantit qu'elles seront exécutées les unes après les autres dans le même environnement de shell.

Cela permet une meilleure gestion des environnements de travail et d'éviter des erreurs comme celle de ne pas pouvoir exécuter des commandes successivement dans différentes cellules.

# 1 . Connexion avec le drive

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2 . Installer la configuration

In [24]:
# Attendre quelque secondes avant de continuer (sync avec le drive)
!mkdir -p /content/drive/MyDrive/SAE_S5

In [25]:
bash_script = """
#!/bin/bash

#####################
#                   #
#       Config      #
#                   #
#####################

apt-get install git

if [ "$#" -ne 3 ]; then
    echo "Usage: $0 <TOKEN> <GITHUB_USER> <GITHUB_EMAIL>"
    exit 1
fi

TOKEN=$1
GITHUB_USER=$2
GITHUB_EMAIL=$3

REPO_DIR="/content/drive/MyDrive/SAE_S5"

if [ -d "$REPO_DIR" ]; then
  echo "Le dossier existe déjà. Suppression du dossier existant..."
  rm -rf "$REPO_DIR"
fi

mkdir -p "$REPO_DIR"

git clone https://$TOKEN@github.com/JagoOgaj/SAE_S5.git "$REPO_DIR"

cd "$REPO_DIR"

git config --global user.name "$GITHUB_USER"
git config --global user.email "$GITHUB_EMAIL"

git remote set-url origin https://$TOKEN@github.com/JagoOgaj/SAE_S5.git

pip install -r "$REPO_DIR/requirements.txt"

echo "Configuration terminée avec succès!"
"""

with open('/content/drive/MyDrive/SAE_S5/config.sh', 'w') as f:
    f.write(bash_script)

print("Le fichier config.sh a été créé avec succès!")

Le fichier config.sh a été créé avec succès!


In [26]:
!chmod +x /content/drive/MyDrive/SAE_S5/config.sh

In [27]:
#!bash /content/drive/MyDrive/SAE_S5/config.sh 

## Git Command

In [30]:
#Use here git command
bash_script = """
#!/bin/bash

#####################
#                   #
#    Git Command    #
#                   #
#####################

REPO_DIR="/content/drive/MyDrive/SAE_S5"

cd "$REPO_DIR"

# Add comand here #
git checkout soheib
"""

with open('/content/drive/MyDrive/SAE_S5/git.sh', 'w') as f:
    f.write(bash_script)


!chmod +x /content/drive/MyDrive/SAE_S5/git.sh

!bash /content/drive/MyDrive/SAE_S5/git.sh

Already on 'soheib'
Your branch is up to date with 'origin/soheib'.


In [31]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Paramètres globaux
buffer_size = 10000  # Nombre d'images à mélanger
batch_size = 32  # Taille de chaque lot d'images
image_size = (64, 64)  # Taille cible des images


In [33]:
from tensorflow.keras.utils import image_dataset_from_directory

# Charger le dataset d'entraînement avec la taille de lot définie directement
train_ds = image_dataset_from_directory(
    directory="/content/celebrities/Dataset/Train",
    class_names=["Female", "Male"],
    image_size=image_size,  # Redimensionner directement à (64, 64)
    batch_size=batch_size  # Appliquer le batch_size ici
)

# Charger le dataset de validation avec la même taille de lot
val_ds = image_dataset_from_directory(
    "/content/celebrities/Dataset/Validation",
    class_names=["Female", "Male"],
    image_size=image_size,
    batch_size=batch_size
)


Found 160000 files belonging to 2 classes.
Found 22598 files belonging to 2 classes.


In [34]:
for images, labels in train_ds.take(1):
    print("Shape of images:", images.shape)  # Doit être (batch_size, 64, 64, 3)
    print("Shape of labels:", labels.shape)  # Doit être (batch_size,)


Shape of images: (32, 64, 64, 3)
Shape of labels: (32,)


In [39]:
def preprocess(image, label, size=(64, 64), mode='train'):
    # Supprime la dimension de lot si elle est présente
    image = tf.squeeze(image, axis=0) if len(image.shape) == 4 else image

    # Redimensionne d'abord l'image pour qu'elle soit suffisamment grande pour le recadrage
    image = tf.image.resize(image, (178, 178))

    if mode == 'train':
        # Appliquer un recadrage aléatoire
        img_crop = tf.image.random_crop(image, size=(178, 178, 3))
        img_resized = tf.image.resize(img_crop, size=size)
        img_flip = tf.image.random_flip_left_right(img_resized)
        return img_flip / 255.0, tf.cast(label, tf.int32)
    else:
        # Recadrage fixe pour la validation/test
        img_crop = tf.image.crop_to_bounding_box(image,
                                                 offset_height=20,
                                                 offset_width=0,
                                                 target_height=178,
                                                 target_width=178)
        img_resized = tf.image.resize(img_crop, size=size)
        return img_resized / 255.0, tf.cast(label, tf.int32)


In [40]:
ds_train = train_ds.map(lambda x, y: preprocess(image=x, label=y, size=image_size, mode='train'))
ds_valid = val_ds.map(lambda x, y: preprocess(image=x, label=y, size=image_size, mode='eval'))


ValueError: in user code:

    File "<ipython-input-38-95b17e4c80a9>", line 2, in None  *
        lambda x, y: preprocess(image=x, label=y, size=image_size, mode='eval')
    File "<ipython-input-39-03ea0936f9c4>", line 16, in preprocess  *
        img_crop = tf.image.crop_to_bounding_box(image,

    ValueError: height must be >= target + offset.
